## 0.0 Entender o problema

-- Objetivo do problema:
    -- 1.0 Previsão do primeiro destino que um novo usuário irá escolher.
    
    --Porque?
      --Qual tipo de modelo de negócio do Airbnb?
        --Marketplace (Conectar pessoas que oferecem acomodação, com as pessoas que estão procurando acomodação).
        
        --Oferta (pessoas oferecendo acomodação)
            --Tamanho do Portfólio.
            -- diversidade/Densidade de Portfólio.
            --Preço médio
        
        --Demanda (pessoa procurando acomodação)
             --Número de Usuários 
             --LTV (LifeTime Value)
             --CAC (Client Aquisition Cost)
        
        Gross Revenue = (Fee * Numero de clientes) - CAC
        
        --Demanda (pessoas procurando acomodação)
        
        
--Proposta da solução:

    --- Modelo de previsão do primeiro destino de um novo usuário.
    
    --- 1.0 Predições e salvar em uma tabela de bando de dados.
    
    ---2.0 Criar uma API
        --- Input: usuario  e suas caracteristicas
        --- Output: usuario e suas caracteristicas com a ** predição do destino**.

        
        

# 0.0 <font color ='red'>Ciclo01: End to End </font>

# 0.0 Imports

In [ ]:
!pip install scikit-plot

In [ ]:
import math
from scipy.stats import chi2_contingency
import scipy.stats as ss
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
import pickle

import warnings

warnings.filterwarnings( 'ignore' )

import sklearn.model_selection
from sklearn import model_selection as ms
from sklearn import preprocessing   as pp
from sklearn import metrics         as m
from scikitplot import metrics      as mt
from IPython.core.display  import HTML
from IPython.core.display  import Image
from tabulate              import tabulate

from imblearn import under_sampling as us
from imblearn import over_sampling  as oversamp
from imblearn import combine        as c

from pandas_profiling import ProfileReport

from keras import models as ml
from keras import layers as l


# 0.1 Helper functions

In [ ]:
def cramer_v(x, y):
    """ calculate Cramers V statistic for categorial-categorial association.
        uses correction from Bergsma and Wicher. 
        """     
    
    cm = pd.crosstab(x, y).values  #.as_matrix()
    n = cm.sum()
    r, k = cm.shape
    
    chi2 = ss.chi2_contingency( cm )[0]
    chi2corr = max( 0, chi2 - (k-1)*(r-1)/(n-1) )
    
    kcorr = k - (k-1)**2/(n-1)
    rcoor = r - (r-1)**2/(n-1)
    return np.sqrt( (chi2corr/n)/ (min(kcorr-1,  rcoor-1)))

def jupyter_settings():
    %matplotlib inline
    %pylab inline
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [25, 12]
    plt.rcParams['font.size'] = 24
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    sns.set()

    
jupyter_settings()


# 0.2 Loading Data

In [ ]:
# datasets source: https://www.kaggle.com/c/airbnb-recruiting-new-user-bookings/data?select=sessions.csv.zip


df_raw = pd.read_csv('dataset/training_users_2.csv', low_memory=True)
df_raw.shape

df_sessions = pd.read_csv('dataset/sessions.csv', low_memory=True)
df_sessions.shape

# 1.0 Data Description

In [ ]:
df1 = df_raw.copy()

## 1.1 Data Dimensions

In [ ]:
print( 'Number of rows: {}'.format(df1.shape[0] ) )
print( 'Number of columns: {}'.format(df1.shape[1] ) )

In [ ]:
print( 'Number of rows: {}'.format(df_sessions.shape[0] ) )
print( 'Number of columns: {}'.format(df_sessions.shape[1] ) )

## 1.2 Data type

In [ ]:
df1.dtypes

In [ ]:
df_sessions.dtypes

## 1.3 NA Check

In [ ]:
 df1.isna().sum() / len( df1 )

In [ ]:
df_sessions.isna().sum() / len(df_sessions)

In [ ]:
#remove missing value complety
#df1 = df1.dropna()

#========================USER===========================================

# date_first_booking
date_first_booking_max = pd.to_datetime( df1['date_first_booking'] ).max().strftime( '%Y-%m-%d' )
df1['date_first_booking'] = df1['date_first_booking'].fillna( date_first_booking_max )


# age
df1 = df1[( df1['age'] > 15 ) & ( df1['age'] < 120 )]
avg_age =  round(df1['age'].mean(),2)
df1['age'] = df1['age'].fillna( avg_age )

# first_affiliate_tracked
df1 = df1[~df1['first_affiliate_tracked'].isna()]

df1['first_affiliate_tracked'].value_counts()

#========================Sessions=========================================
#user_id  - 3%
df_sessions= df_sessions[~df_sessions['user_id'].isna()]

#action - 7%
df_sessions= df_sessions[~df_sessions['action'].isna()]

#action_type - 10$
df_sessions= df_sessions[~df_sessions['action_type'].isna()]

#action_detail- 10%
df_sessions= df_sessions[~df_sessions['action_detail'].isna()]

#device_type - 0%
df_sessions= df_sessions[~df_sessions['device_type'].isna()]

#secs_elapsed - 0%
df_sessions= df_sessions[~df_sessions['secs_elapsed'].isna()]



In [ ]:
df_sessions.isna().sum() / len(df_sessions)

In [ ]:
df1.isna().sum() / len(df1)

## 1.4 Change Data Type

In [ ]:
# date_account_created    
df1["date_account_created"] = pd.to_datetime( df1["date_account_created"] )

# timestamp_first_active   
df1["timestamp_first_active"] = pd.to_datetime( df1["timestamp_first_active"], format='%Y%m%d%H%M%S' )

# date_first_booking       
df1["date_first_booking"] = pd.to_datetime( df1["date_first_booking"] )

# age       
df1["age"] = df1["age"].astype( int )

In [ ]:
df1.dtypes

## 1.5 Check Balanced Data

In [ ]:
df1["country_destination"].value_counts(normalize=True)

## 1.6 Descriptive Analysis

## 1.6 General

In [ ]:
# Users
num_attributes = df1.select_dtypes( include=['int32','int64', 'float64'])
cat_attributes = df1.select_dtypes( exclude=['int32', 'int64', 'float64','datetime64[ns]'])
time_attributes = df1.select_dtypes( include=['datetime64[ns]'])

#Sessions
num_attributes_sessions = df_sessions.select_dtypes( include=['int32','int64', 'float64'])
cat_attributes_sessions = df_sessions.select_dtypes( exclude=['int64', 'float64','datetime64[ns]'])
time_attributes_sessions = df_sessions.select_dtypes( include=['datetime64[ns]'])

## 1.6.1 Numeral - Users

In [ ]:
# Central Tendency - Mean, Median
ct1 = pd.DataFrame(num_attributes.apply(np.mean)).T
ct2 = pd.DataFrame(num_attributes.apply(np.median)).T

# Dispersions - Std, Min, Max, Range, Skew, Kurtosis
d1 = pd.DataFrame(num_attributes.apply(np.std)).T
d2 = pd.DataFrame(num_attributes.apply(min)).T
d3 = pd.DataFrame(num_attributes.apply(max)).T
d4 = pd.DataFrame(num_attributes.apply( lambda x: x.max() - x.min())).T
d5 = pd.DataFrame(num_attributes.apply(lambda x: x.skew())).T
d6 = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis())).T

# concatenar 
ct = pd.concat([d2, d3, d4,ct1, ct2, d1, d5, d6]).T.reset_index()
ct.columns = ['attributes','min','max', 'range','mean','median','std','skew','kurtosis']
ct

## 1.6.2 Numeral - Sessions

In [ ]:
# Central Tendency - Mean, Median
ct1 = pd.DataFrame(num_attributes_sessions.apply(np.mean)).T
ct2 = pd.DataFrame(num_attributes_sessions.apply(np.median)).T

# Dispersions - Std, Min, Max, Range, Skew, Kurtosis
d1 = pd.DataFrame(num_attributes_sessions.apply(np.std)).T
d2 = pd.DataFrame(num_attributes_sessions.apply(min)).T
d3 = pd.DataFrame(num_attributes_sessions.apply(max)).T
d4 = pd.DataFrame(num_attributes_sessions.apply( lambda x: x.max() - x.min())).T
d5 = pd.DataFrame(num_attributes_sessions.apply(lambda x: x.skew())).T
d6 = pd.DataFrame(num_attributes_sessions.apply(lambda x: x.kurtosis())).T

# concatenar 
ct = pd.concat([d2, d3, d4,ct1, ct2, d1, d5, d6]).T.reset_index()
ct.columns = ['attributes','min','max', 'range','mean','median','std','skew','kurtosis']
ct

## 1.6.2 Categorical - Users

In [ ]:
cat_attributes.drop('id', axis=1).describe()

## 1.6.3 Categorical - Sessions

In [ ]:
cat_attributes_sessions.drop('user_id', axis=1).describe()

## 1.6.2 Categorical - Sessions

In [ ]:
# list of attributes for Cramer's V correlation
cat_attributes_list = cat_attributes_sessions.drop('user_id', axis=1).columns.tolist()

coor_dict = {}

for i in range(len(cat_attributes_list)):
    corr_list =  []
    for j in range(len(cat_attributes_list)):
        ref = cat_attributes_list[i]
        feat=cat_attributes_list[j]
        
        # correlation
        corr = cramer_v(cat_attributes_sessions[ref], cat_attributes_sessions[feat])
        # append a list
        corr_list.append(corr)
    # append a correlation list for each atributes   
    coor_dict[ref] = corr_list

In [ ]:
d = pd.DataFrame(coor_dict)
d = d.set_index(d.columns)
sns.heatmap(d, annot=True);

# 2.0 Feature Engineering

In [ ]:
df2 = df1.copy()
df_sessions2 = df_sessions.copy()

In [ ]:
df2.dtypes

## 2.1 Create New Features

In [ ]:
# days from first active up to first booking
df2['first_active'] = pd.to_datetime( df2['timestamp_first_active'].dt.strftime( '%Y-%m-%d' ) )
df2['days_from_frist_active_until_booking'] = ( df2['date_first_booking'] - df2['first_active'] ).apply( lambda x: x.days )

# days from first active upt to account created
df2['days_from_first_active_until_account_created'] = ( df2['date_account_created'] - df2['first_active'] ).apply( lambda x: x.days )

# days from account createad up to first booking
df2['days_from_account_created_until_first_booking'] = ( df2['date_first_booking'] - df2['date_account_created'] ).apply( lambda x: x.days )

# ================== First Active ==================
# year first active
df2['year_first_active'] = df2['first_active'].dt.year

# month first active
df2['month_fist_active'] = df2['first_active'].dt.month

# day first active
df2['day_first_active'] = df2['first_active'].dt.day

# day of week first active
df2['day_of_week_first_active'] = df2['first_active'].dt.dayofweek

# week of year first active
df2['week_of_year_first_active'] = df2['first_active'].dt.weekofyear

# ================== First Booking ==================
# year first booking
df2['year_first_booking'] = df2['date_first_booking'].dt.year

# month first booking
df2['month_first_booking'] = df2['date_first_booking'].dt.month

# day first booking
df2['day_first_booking'] = df2['date_first_booking'].dt.day

# day of week first booking
df2['day_of_week_first_booking'] = df2['date_first_booking'].dt.dayofweek

# week of year first booking
df2['week_of_year_first_booking'] = df2['date_first_booking'].dt.weekofyear

# ================== First Account Created =================
# year first booking
df2['year_account_created'] = df2['date_account_created'].dt.year

# month  account_created
df2['month_account_created'] = df2['date_account_created'].dt.month

# day  account_created
df2['day_account_created'] = df2['date_account_created'].dt.day

# day of week  account_created
df2['day_of_week_account_created'] = df2['date_account_created'].dt.dayofweek

# week of year  account_created
df2['week_of_year_account_created'] = df2['date_account_created'].dt.weekofyear


# 3.0 Data filtering

In [ ]:
df3 = df2.copy()
df_sessions3 = df_sessions2.copy()

## 3.1 Filtering Rows

In [ ]:
# Filtering rows:
# age - greater than 15 and lower than 120 - There are few people over 12O year old   
df3 = df3[( df3['age'] > 15 ) & ( df3['age'] < 120 )]

# secs_elapsed - there is no possible 0 secs elapsed on website
#df3 = df3[df3['secs_elapsed'] > 0]

## 3.2 Columns Selection

In [ ]:
# delete original datetime and high correlation
cols = ['date_account_created', 'date_account_created', 'date_first_booking', 'timestamp_first_active', 'first_active', # original datetime
        'days_from_first_active_until_booking',  'year_first_active',  'month_first_active', 'day_first_active',
        'day_of_week_first_active', 'week_of_year_first_active', 'month_first_booking', 'month_account_created',
        'year_first_booking', 'week_of_year_first_booking', 'affiliate_provider',
        'first_browser', 'first_device_type'] # high correlation

df3 = df3.drop( cols, axis=1 )

# 4.0 Balanced Dataset

In [ ]:
df4 = df3.copy()

In [ ]:
 # Encoder Categorical Variables
ohe = pp.OneHotEncoder()

# Numerical
col_num = df4.select_dtypes( include=['int64', 'float64'] ).columns.tolist()

# Categorical
col_cat = df4.select_dtypes( exclude=['int64', 'float64', 'datetime64[ns]'] ).drop( ['id', 'country_destination'], axis=1 ).columns.tolist()

# encoding
df4_dummy = pd.DataFrame( ohe.fit_transform( df4[ col_cat] ).toarray(), index=df4.index )

# join numerical and categorical
df42 = pd.concat( [df4[col_num], df4_dummy], axis=1 )
df42.shape

##  4.1 Random Undersampling

In [ ]:
# ratio_balanced
ratio_balanced = {'NDF': 10000 }
# define sampler
undersampling = us.RandomUnderSampler( sampling_strategy=ratio_balanced, random_state=32 )

# apply sampler
X_under, y_under = undersampling.fit_resample( df42, df4['country_destination'] )

In [ ]:
df4['country_destination'].value_counts()

In [ ]:
y_under.value_counts()

 ## 4.2 Random Oversampling

In [ ]:
# ratio_balanced
#ratio_balanced = {'NDF': 10000 }

# define sampler
oversampling = oversamp.RandomOverSampler( sampling_strategy='all', random_state=32 )

# apply sampler
X_over, y_over = oversampling.fit_resample( df42, df4['country_destination'] )

In [ ]:
y_over.value_counts()

In [ ]:
df4['country_destination'].value_counts()

 ## 4.3 SMOTE + TOMELINK

In [ ]:
#df4['country_destination'].value_counts().to_dict()

ratio_balanced ={'NDF': 54852,
                 'US': 48057,
                 'other': 6*7511,
                 'FR': 12*3669,
                 'IT': 20*2014,
                 'GB': 30*1758,
                 'ES': 30*1685,
                 'CA': 40*1064,
                 'DE': 45*841,
                 'NL': 80*595,
                 'AU': 85*433,
                 'PT': 300*157}



In [ ]:
# define sampler
smt = c.SMOTETomek( sampling_strategy=ratio_balanced, random_state=32, n_jobs=-1 )

# apply sampler
X_smt, y_smt = smt.fit_resample( df42, df4['country_destination'] )


In [ ]:
# numerical data
df43 = X_smt[col_num]

# categorical data
df44 = X_smt.drop(col_num, axis=1)
df45 = pd.DataFrame(ohe.inverse_transform(df44), columns=col_cat, index=df44.index)

# join numerical categorical 
df46 = pd.concat([df43,df45], axis=1)
df46['country_destination'] =  y_smt

# 5.0 Exploratory Data Analysis

## 5.1 Univariate Analysis - Feature Behavior (Balanced Dataset)

In [ ]:
df51 = df46.copy()

In [ ]:
proof = ProfileReport(df51)
proof.to_notebook_iframe()

## 5.2 Bivariate Analysis - Feature Behavior (Unbalanced Dataset)

**H0.** Os usuários levem até 3 dias, em média, para fazer o cadastro no site em todos os destinos.

**Verdadeira.** Os usuários levam até 3 dias, em média para realizar o cadastro no site em todos os destinos

In [ ]:
plt.figure( figsize=(25, 12))
aux01 = df5[['days_from_first_active_until_account_created', 'country_destination']].groupby( 'country_destination' ).mean().reset_index()
sns.barplot( x='country_destination', y='days_from_first_active_until_account_created' , data=aux01 );
plt.ylabel( 'Average days until Accout Creation');

**H1.** O numero de reservas do Airbnb cresce ou decresce ao longo do tempo?
**Depende.**

In [ ]:
plt.figure( figsize=(25,12))
aux01 = df5[df5['country_destination'] != 'NDF']
aux01 = aux01[['year_first_booking', 'month_first_booking', 'country_destination']]\
                .groupby( ['year_first_booking', 'month_first_booking'] )\
                .count() \
                .reset_index()

aux01['year-month'] = aux01.apply( lambda x: str( x['year_first_booking'] ) + '-' + str( x['month_first_booking'] ), axis=1 )

sns.barplot( x='year-month', y='country_destination', data=aux01 );
plt.xticks( rotation=90 );

**H2.** O numero de reservas total cresce 10% ao ano em todos os anos.
**Falsa.** O numero de reservas total cresce 10% ao ao apenas entre 2011 e 2013.

# <font color='red'> 5.0. Data Preparation

In [ ]:
df5 = df4.copy()

In [ ]:
# dummy variable
df5_dummy = pd.get_dummies( df5.drop( ['id', 'country_destination'], axis=1 ) )

# join id and country destination
df5 = pd.concat( [df5[['id', 'country_destination']], df5_dummy], axis=1 )

In [ ]:
df5.shape

# 6.0 Feature Selection

In [ ]:
cols_drop = ["date_account_created", "timestamp_first_active","date_first_booking", 'first_active'] # original dates
df6 = df5.drop(cols_drop, axis=1)

In [ ]:
X = df6.drop('country_destination', axis=1)
y = df6['country_destination'].copy()

In [ ]:
# Split dataset into training and test
X_train, X_test, y_train, y_test = ms.train_test_split( X, y, test_size=0.2, random_state=32 )

# 7.0 Machine Learning - Neural Network - MLP

In [ ]:
x_train = X_train.drop("id", axis=1)
x_test = X_test.drop("id", axis=1)

## 7.1. Baseline Model 

In [ ]:
country_destination_list = df1['country_destination'].drop_duplicates().sort_values().tolist()
k_num = y_test.shape[0]
# adding the weights according to column distribution "country_destination"
country_destination_weights= df1['country_destination'].value_counts(normalize=True).sort_index().tolist()
# random model

yhat_random = random.choices(population=country_destination_list,
                             weights=country_destination_weights,              
                             k=k_num)

## 7.1.1 Baseline Model Performance

In [ ]:
# Acurracy
acc_random = m.accuracy_score(y_test, yhat_random )
print("Accuracy: {}".format(acc_random))

# Balanced accuracy
balanced_acc_random = m.balanced_accuracy_score(y_test, yhat_random)
print("Balanced Accuracy: {}".format(balanced_acc_random))

#Kappa Metrics
Kappa_random = m.cohen_kappa_score(y_test, yhat_random)
print("Kappa Score: {}".format(balanced_acc_random))

#Classification report
print(m.classification_report(y_test, yhat_random))


# Confusion Matrix
mt.plot_confusion_matrix( y_test, yhat_random, normalize=False, figsize=(12,12) )

# 7.2 Neural Network - MLP

In [ ]:
# One Hot Encoder
ohe = pp.OneHotEncoder()
y_train_nn = ohe.fit_transform( y_train.values.reshape(-1, 1) ).toarray()

In [ ]:
y_train_nn

In [ ]:
x_train.shape

In [ ]:
# model definition
model = ml.Sequential()
model.add(l.Dense( 256,input_dim=x_train.shape[1], activation='relu') )
model.add(l.Dense(12, activation="softmax") )

# model compile
model.compile( loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"] )

model.fit( x_train, y_train_nn, epochs=100, verbose=0)

In [ ]:
# prediction
pred_nn = model.predict( x_test )

# invert prediction
yhat_nn = ohe.inverse_transform( pred_nn)

# prediction prepare
y_test_nn = y_test.to_numpy()
yhat_nn = yhat_nn.reshape(1, -1)[0]

## 7.2.2 NN Performance

In [ ]:
# Acurracy
acc_nn = m.accuracy_score(y_test_nn, yhat_nn )
print("Accuracy: {}".format(acc_nn))

# Balanced accuracy
balanced_acc_nn = m.balanced_accuracy_score(y_test_nn, yhat_nn)
print("Balanced Accuracy: {}".format(balanced_acc_nn))

#Kappa Metrics
Kappa_nn = m.cohen_kappa_score(y_test_nn, yhat_nn)
print("Kappa Score: {}".format(Kappa_nn))

#Classification report
print(m.classification_report(y_test_nn, yhat_nn))


# Confusion Matrix
mt.plot_confusion_matrix( y_test_nn, yhat_nn, normalize=False, figsize=(12,12) )
#

## 7.2.3 NN Performance - Cross Validation

In [ ]:
# generate k-fold
num_folds = 5
kfold = ms.StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=32)

balanced_acc_nn_list =[]
Kappa_nn_list = []
i = 1
for train_ix, val_ix in kfold.split(x_train, y_train):
    print("fold number: {}/{}".format(i, num_folds))
          
    # get fold
    x_train_fold = x_train.iloc[train_ix]
    y_train_fold = y_train.iloc[train_ix]
    
    x_val_fold = x_train.iloc[val_ix]
    y_val_fold = y_train.iloc[val_ix]
    
    # target hot-encoding --> categorical to numeral
    ohe = pp.OneHotEncoder()
    y_train_fold_nn = ohe.fit_transform(y_train_fold.values.reshape(-1, 1)).toarray()

    # model defition
    model = ml.Sequential()
    model.add(l.Dense(256, input_dim=x_train.shape[1], activation="relu"))
    model.add(l.Dense(12, activation="softmax"))

    # compile model
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

    # training model
    model.fit(x_train_fold, y_train_fold_nn, epochs=100, batch_size=32, verbose=0)

    # prediction
    pred_nn = model.predict(x_val_fold)
    yhat_nn = ohe.inverse_transform(pred_nn) # numeral to categorical


    # prepare data
    y_test_nn = y_val_fold.to_numpy()
    Yhat_nn = yhat_nn.reshape(-1, 1)[0]

    # metrics

    # Balanced accuracy
    balanced_acc_nn = m.balanced_accuracy_score(y_test_nn, yhat_nn)
    balanced_acc_nn_list.append(balanced_acc_nn)
    #print("Balanced Accuracy: {}".format(balanced_acc_nn))

    #Kappa Metrics
    Kappa_nn = m.cohen_kappa_score(y_test_nn, yhat_nn)
    Kappa_nn_list.append(Kappa_nn)
    #print("Kappa Score: {}".format(Kappa_nn))

    i += 1

In [ ]:
print("avg Balanced Accuracy:  {} +/- {}".format(round(np.mean(balanced_acc_nn_list), 4),
                                                np.round(np.std(balanced_acc_nn_list), 4)))


In [ ]:
print("avg Kappa score: {} +/- {}".format(np.round(np.mean(Kappa_nn_list), 4),
                                          np.round(np.std(Kappa_nn_list), 4)))